In [14]:
import torch
import math

import timm
from fastai.vision.all import *

if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    print("Found mps")
    default_device(torch.device("mps"))
else:
    print("Using cpu")
    default_device(torch.device("mps"))

Using cpu


In [15]:
path = untar_data(URLs.MNIST)
print(path)

/Users/gwilliams/.fastai/data/mnist_png


In [16]:
block = DataBlock(
        blocks=(ImageBlock, CategoryBlock),
        get_items=get_image_files,
        splitter=RandomSplitter(valid_pct=0.2, seed=42),
        get_y=parent_label)
#        batch_tfms=aug_transforms(mult=2., do_flip=False))

In [17]:
loaders = block.dataloaders(path/"training")

Could not do one pass in your dataloader, there is something wrong in it. Please see the stack trace below:


RuntimeError: The MPS backend is supported on MacOS 12.3+.Current OS version can be queried using `sw_vers`

In [ ]:
loaders.train.show_batch(max_n=4, nrows=1)

In [ ]:
# squeezenet default fast ai learner
#learn = vision_learner(loaders, squeezenet1_0, loss_func=nn.CrossEntropyLoss(), metrics=accuracy)
#learn.summary()

In [ ]:
class Mnist_NN(nn.Module):
    def __init__(self,pretrained):
        super().__init__()
        self.lin1 = nn.Linear(784, 512, bias=True) 
        self.lin2 = nn.Linear(512, 256, bias=True)
        self.lin3 = nn.Linear(256, 10, bias=True)

    def forward(self, xb):
        x = xb.view(-1,784) 
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        return self.lin3(x)

In [ ]:
def get_model(model, pretrained=False, progress=True, **kwargs):
    """model: function to load the model, e.g. resnet18
        pretrained, progress: to be passed to the model function
    """
    m = model(pretrained=pretrained, progress=progress, **kwargs) # loads standard model
    ##m.avgpool = nn.AdaptiveAvgPool2d(output_size=(100,100)) # changes one layer
    return m

In [ ]:

from fastai.vision.learner import _update_first_layer
def create_timm_body(arch:str, pretrained=True, cut=None, n_in=3):
    "Creates a body from any model in the `timm` library."
    try:
        model = create_model(arch, pretrained=pretrained, num_classes=0, global_pool='')
        print(arch, type(model),sum([p.numel() for p in model.parameters()]))
    
        _update_first_layer(model, n_in, pretrained)
        if cut is None:
            ll = list(enumerate(model.children()))
            cut = next(i for i,o in reversed(ll) if has_pool_type(o))
        if isinstance(cut, int): return nn.Sequential(*list(model.children())[:cut])
        elif callable(cut): return cut(model)
        else: raise NamedError("cut must be either integer or function")
    except:
        print("error processing cut")

In [ ]:
body = create_timm_body('mobilenetv3_small_050', pretrained=True)

In [ ]:
nf = num_features_model(nn.Sequential(*body.children())); nf


In [ ]:
head = create_head(nf, loaders.c)

In [ ]:
model = nn.Sequential(body, head)

In [ ]:
apply_init(model[1], nn.init.kaiming_normal_)

In [ ]:
len(model)

In [ ]:
def get_model(model, pretrained=False, progress=True, **kwargs):
    """model: function to load the model, e.g. resnet18
        pretrained, progress: to be passed to the model function
    """
    m = model(pretrained=pretrained, progress=progress, **kwargs) # loads standard model
    ##m.avgpool = nn.AdaptiveAvgPool2d(output_size=(100,100)) # changes one layer
    return m

In [ ]:
##learn = vision_learner(loaders, partial(get_model, model=model), loss_func=nn.CrossEntropyLoss(), metrics=accuracy)
learn = vision_learner(loaders, squeezenet1_0, loss_func=nn.CrossEntropyLoss(), metrics=accuracy)

##learn = Learner(loaders, model, loss_func=LabelSmoothingCrossEntropy())


In [ ]:
learn.summary()

In [ ]:
learn.fit_one_cycle(1)